In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename

In [135]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count('DT') 
#                or path.count('day_no') 
               or path.count('Amt')
               or path.count('Fraud') 
               or path.count('D')
               or path.count('C')
               or path.count('V')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('Reg')
#                or path.count('P_email')
#                or path.count('R_email')
#                or path.count('M')
#                or path.count('Product')
              ]
test_paths = [path for path in test_paths 
               if path.count('DT') 
#                or path.count('day_no') 
               or path.count('Amt')
               or path.count('Fraud') 
               or path.count('D')
               or path.count('C')
               or path.count('V')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('Reg')
#                or path.count('P_email')
#                or path.count('R_email')
#                or path.count('M')
#                or path.count('Product')
              ]

train_df = parallel_load_data(train_paths)
test_df = parallel_load_data(test_paths)

In [31]:
train_df.head()

,D15,V7,V210,V225,V203,V3,V61,V252,TransactionID,V170,...,V304,V272,D13,V267,V303,TransactionAmt,V91,D8,V2,ProductCD
0,0.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,2987000,NaN,...,0.0,NaN,NaN,NaN,0.0,68.5,0.0,NaN,1.0,W
1,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2987001,NaN,...,0.0,NaN,NaN,NaN,0.0,29.0,0.0,NaN,NaN,W
2,315.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,2987002,NaN,...,0.0,NaN,NaN,NaN,0.0,59.0,0.0,NaN,1.0,W
3,111.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2987003,NaN,...,0.0,NaN,NaN,NaN,0.0,50.0,0.0,NaN,NaN,W
4,NaN,NaN,0.0,0.0,0.0,NaN,NaN,1.0,2987004,1.0,...,1.0,0.0,NaN,0.0,1.0,50.0,NaN,NaN,NaN,H


In [32]:
train_df.set_index(COLUMN_ID, inplace=True)
list_path = glob('../output/0830_ieee__same_user__pattern-user_keys__card*')
for path in list_path:
    fname = path[56:].replace('csv', '').replace(r'.', '')
    same_user = pd.read_csv(path).set_index(COLUMN_ID)
    train_df[f'same_user__{fname}']  = same_user['same_user_id']

In [38]:
startdate = datetime.datetime(2017,11,4)
train_df['datetime'] = train_df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
train_df['month'] = train_df['datetime'].map(lambda x: x.month)
train_df['date'] = train_df['datetime'].map(lambda x: x.date())

In [45]:
cols_V = sorted([int(col[1:]) for col in train_df.columns if col.startswith('V')])
cols_V = ['V' + str(col) for col in cols_V]
cols_D = sorted([int(col[1:]) for col in train_df.columns if col.startswith('D')])
cols_D = ['D' + str(col) for col in cols_D]
cols_C = sorted([int(col[1:]) for col in train_df.columns if col.startswith('C')])
cols_C = ['C' + str(col) for col in cols_C]
cols_same = sorted([col for col in train_df.columns if col.startswith('same')])
cols_trans = sorted([col for col in train_df.columns if col.startswith('Trans')])
cols_sort = cols_same + ['datetime', COLUMN_TARGET, 'ProductCD'] + cols_trans + cols_D + cols_C + cols_V

セール時期だと、ProductCD HとRの割合が2.5倍になる。
それに伴い、HとRのFraud率は0.5倍になる

In [1]:
# fraud ratio
COLUMN_PRO = 'ProductCD'
tmp = train_df[train_df['month']>10]
print(tmp.groupby('ProductCD')[COLUMN_TARGET].mean())
tmp = train_df[train_df['month']<=10]
print(tmp.groupby('ProductCD')[COLUMN_TARGET].mean())

NameError: name 'train_df' is not defined

In [ ]:
# row count
print(tmp[COLUMN_PRO].value_counts() / tmp[COLUMN_PRO].value_counts().sum())
tmp = train_df[train_df['month']<=10]
print(tmp[COLUMN_PRO].value_counts() / tmp[COLUMN_PRO].value_counts().sum())

In [136]:
pd.set_option('max_rows', 300)
list_product = ['W', 'C', 'S', 'H', 'R']
df_product_C = pd.DataFrame(index=cols_C)

all_data = pd.concat([train_df[['ProductCD'] + cols_C], test_df[['ProductCD'] + cols_C]], axis=0)

for product in list_product:
    tmp = all_data[all_data['ProductCD']==product]
    tmp = tmp[cols_C].max().to_frame(f'max_ProductCD-{product}')
    df_product_C = df_product_C.join(tmp, how='left')
df_product_C

,max_ProductCD-W,max_ProductCD-C,max_ProductCD-S,max_ProductCD-H,max_ProductCD-R
C1,674.0,4685.0,39.0,39.0,1505.0
C2,676.0,5691.0,38.0,44.0,1672.0
C4,0.0,2253.0,29.0,30.0,1107.0
C5,376.0,0.0,0.0,0.0,0.0
C6,506.0,2253.0,5.0,22.0,1147.0
C7,0.0,2255.0,0.0,0.0,0.0
C8,0.0,3331.0,63.0,25.0,588.0
C9,572.0,0.0,0.0,0.0,0.0
C10,0.0,3257.0,223.0,33.0,542.0
C11,596.0,3188.0,34.0,26.0,1210.0


In [ ]:
df_product_V = pd.DataFrame(index=cols_V)

for product in list_product:
    tmp = all_data[all_data['ProductCD']==product]
    tmp = tmp[cols_V].max().to_frame(f'max_ProductCD-{product}')
    df_product_V = df_product_V.join(tmp, how='left')
df_product_V